# Red Neuronal Recurrente (GRU) para generar una melodía
### Instalar midi_player y music21 (pip install midi_player music21)

In [12]:
import os, glob
import random

import numpy as np 
import pandas as pd
import tensorflow as tf

from midi_player import MIDIPlayer
from music21 import converter, instrument, note, chord, stream

from tensorflow.data import Dataset
from tensorflow.keras import losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

## Leer un archivo de ejemplo

In [13]:
midi_file = 'datasets/mozart/mz_311_1.mid'  # Archivo MIDI de ejemplo
MIDIPlayer(midi_file, 460)  # Reproducir el archivo MIDI

### Extraer la notas de todas la melodías (archivos .mid)

In [14]:
notes = []  # Lista para almacenar las notas extraídas

for file in glob.glob("datasets/mozart/*.mid"):
    midi = converter.parse(file)  
    
    songs = instrument.partitionByInstrument(midi) 

    for part in songs.parts:
        pick = part.recurse()  
    for element in pick:
        if isinstance(element, note.Note):  
            notes.append(str(element.pitch))  
        elif isinstance(element, chord.Chord):  
            notes.append('.'.join(str(n) for n in element.normalOrder)) 
print(notes)

[]


### Crear un vocabulario de todas las notas encontradas

In [15]:
vocab = sorted(set(notes))
note2idx = {n: i for i, n in enumerate(vocab)}
idx2note = np.array(vocab)

print('Vocabulario: ', len(vocab), ' notas únicas')
print(f'Notas: {vocab}')

Vocabulario:  0  notas únicas
Notas: []


### Crear secuencias temporales

In [16]:
seq_length = 128
char_dataset = Dataset.from_tensor_slices(encoded)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

BATCH_SIZE = 128
BUFFER_SIZE = 1000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

NameError: name 'encoded' is not defined

### Construir la red neuronal con capas GRU

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return Sequential([
        Input(batch_shape=(batch_size, None)),
        Embedding(vocab_size, embedding_dim),
        GRU(rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'),
        GRU(rnn_units // 2, return_sequences=True, stateful=True,
            recurrent_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])